## Package

In [2]:
import os,cv2,keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import os.path
from pathlib import Path

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
from tensorflow.keras.models import Sequential
from keras.models import Model,load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split

from keras.utils import plot_model
from keras import regularizers
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
#from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import glorot_uniform
import visualkeras
from keras.optimizers import Adam

from IPython.display import Markdown, display
import seaborn as sns

# from lib import proc_img
# from lib import create_gen

def printmd(string):
    #Print with Markdowns    
    display(Markdown(string))

time: 2.47 s


In [ ]:
%load_ext autotime
RANDOM_STATE = 42

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

time: 47.2 ms


In [4]:
seed = 7
np.random.seed(seed)

time: 86.9 ms


In [5]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

time: 70.7 ms


In [6]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

time: 605 ms


In [7]:
tf.test.is_built_with_cuda()

True

time: 72.4 ms


In [8]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

time: 1.38 s


In [9]:
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

time: 2.53 ms


In [10]:
def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

time: 134 ms


In [11]:
dense_sizes_1 = [64,128,256,512]
dense_sizes_2 = [64,128,256,512]
learning_rates_1 = [1e-1,1e-2, 1e-3, 1e-4, 1e-5]

s1_filter_sizes_1 = [16,32,64,128,256,512]

s2_filter_sizes_1 = [16,32,64,128,256,512]
s2_filter_sizes_2 = [16,32,64,128,256,512]
s2_filter_sizes_3 = [128,256,512,1024,2048]

s3_filter_sizes_1 = [16,32,64,128,256,512]
s3_filter_sizes_2 = [16,32,64,128,256,512]
s3_filter_sizes_3 = [128,256,512,1024,2048]

s4_filter_sizes_1 = [16,32,64,128,256,512]
s4_filter_sizes_2 = [16,32,64,128,256,512]
s4_filter_sizes_3 = [128,256,512,1024,2048]

s5_filter_sizes_1 = [16,32,64,128,256,512]
s5_filter_sizes_2 = [16,32,64,128,256,512]
s5_filter_sizes_3 = [128,256,512,1024,2048]

dp_sizes_1 = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]
dp_sizes_2 = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]

wd_sizes_1=[0.0,0.1, 0.001, 0.0001]
wd_sizes_2=[0.0,0.1, 0.001, 0.0001]

opt_layer_1=0
opt_layer_2=0
opt_layer_3=0
opt_layer_4=0
opt_layer_5=0
opt_layer_6=0
opt_layer_7=0
opt_layer_8=0
opt_layer_9=0
opt_layer_10=0
opt_layer_11=0
opt_layer_12=0
opt_layer_13=0
opt_layer_14=0
opt_layer_15=0
opt_layer_16=0
opt_layer_17=0
opt_layer_18=0
opt_layer_19=0
opt_layer_20=0

temp=0
test_acc=0
combinations=0
flag=0
for wd_size_2 in wd_sizes_2 :
    for wd_size_1 in wd_sizes_1 :
        for dp_size_2 in dp_sizes_2 :
            for dp_size_1 in dp_sizes_1 :
                for s5_filter_size_3 in s5_filter_sizes_3:
                    for s5_filter_size_2 in s5_filter_sizes_2:
                        for s5_filter_size_1 in s5_filter_sizes_1:
                            for s4_filter_size_3 in s4_filter_sizes_3:
                                for s4_filter_size_2 in s4_filter_sizes_2:
                                    for s4_filter_size_1 in s4_filter_sizes_1 :
                                        for s3_filter_size_3 in s3_filter_sizes_3:
                                            for s3_filter_size_2 in s3_filter_sizes_2:
                                                for s3_filter_size_1 in s3_filter_sizes_1:
                                                    for s2_filter_size_3 in s2_filter_sizes_3:
                                                        for s2_filter_size_2 in s2_filter_sizes_2:
                                                            for s2_filter_size_1 in s2_filter_sizes_1:
                                                                for s1_filter_size_1 in s1_filter_sizes_1:
                                                                    for dense_size_2 in dense_sizes_2:
                                                                        for dense_size_1 in dense_sizes_1:
                                                                            for learning_rate_1 in learning_rates_1:
                                                                                print(learning_rate_1,dense_size_1,dense_size_2,s1_filter_size_1,s2_filter_size_1,s2_filter_size_2,s2_filter_size_3,s3_filter_size_1,s3_filter_size_2,s3_filter_size_3,s4_filter_size_1,s4_filter_size_2,s4_filter_size_3,s5_filter_size_1,s5_filter_size_2,s5_filter_size_3,dp_size_1,dp_size_2,wd_size_1,wd_size_2)
                                                                                input_shape=(32, 32, 3)
                                                                                X_input = Input(input_shape)

                                                                                X = ZeroPadding2D((1, 1))(X_input)

                                                                                X = Conv2D(s1_filter_size_1, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
                                                                                X = BatchNormalization(axis=3, name='bn_conv1')(X)
                                                                                X = Activation('relu')(X)
                                                                                X = MaxPooling2D((3, 3), strides=(2, 2))(X)

                                                                                X = convolutional_block(X, f=3, filters=[s2_filter_size_1, s2_filter_size_2, s2_filter_size_3], stage=2, block='a', s=1)
                                                                                X = identity_block(X, 3, [s2_filter_size_1, s2_filter_size_2, s2_filter_size_3], stage=2, block='b')
                                                                                X = identity_block(X, 3, [s2_filter_size_1, s2_filter_size_2, s2_filter_size_3], stage=2, block='c')

                                                                                X = convolutional_block(X, f=3, filters=[s3_filter_size_1, s3_filter_size_2, s3_filter_size_3], stage=3, block='a', s=2)
                                                                                X = identity_block(X, 3, [s3_filter_size_1, s3_filter_size_2, s3_filter_size_3], stage=3, block='b')
                                                                                X = identity_block(X, 3, [s3_filter_size_1, s3_filter_size_2, s3_filter_size_3], stage=3, block='c')
                                                                                X = identity_block(X, 3, [s3_filter_size_1, s3_filter_size_2, s3_filter_size_3], stage=3, block='d')

                                                                                X = convolutional_block(X, f=3, filters=[s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='a', s=2)
                                                                                X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='b')
                                                                                X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='c')
                                                                                X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='d')
                                                                                X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='e')
                                                                                X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='f')

                                                                                X = X = convolutional_block(X, f=3, filters=[s5_filter_size_1, s5_filter_size_2, s5_filter_size_3], stage=5, block='a', s=2)
                                                                                X = identity_block(X, 3, [s5_filter_size_1, s5_filter_size_2, s5_filter_size_3], stage=5, block='b')
                                                                                X = identity_block(X, 3, [s5_filter_size_1, s5_filter_size_2, s5_filter_size_3], stage=5, block='c')

                                                                                X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)

                                                                                X= Dropout(dp_size_1)(X)
                                                                                X = Flatten()(X)
                                                                                X=Dense(dense_size_1, activation='relu',kernel_regularizer=regularizers.l2(wd_size_1), name='fc1',kernel_initializer=glorot_uniform(seed=0))(X)
                                                                                X=Dense(dense_size_2, activation='relu',kernel_regularizer=regularizers.l2(wd_size_2), name='fc2',kernel_initializer=glorot_uniform(seed=0))(X)
                                                                                X= Dropout(dp_size_2)(X)
                                                                                X = Dense(10,activation='softmax', name='fc3',kernel_initializer=glorot_uniform(seed=0))(X)

                                                                                model = Model(inputs=X_input, outputs=X, name='ResNet50')

                                                                                my_callbacks = [ tf.keras.callbacks.EarlyStopping(patience=3)]
                                                                                opt = Adam(learning_rate=learning_rate_1)

                                                                                model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                                                                                                                optimizer=opt,
                                                                                                                metrics=['accuracy'])
                                                                                hist=model.fit(train_images,train_labels,
                                                                                          batch_size=64,
                                                                                          epochs=100,
                                                                                          validation_split=0.2,
                                                                                          callbacks=[my_callbacks])

                                                                                test_acc = hist.history["val_accuracy"][0]
                                                                                print(test_acc,temp)
                                                                                combinations=combinations+1
                                                                                if flag==0 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_1 = learning_rate_1
                                                                                        temp=test_acc
                                                                                    learning_rates_1 = [opt_layer_1]

                                                                                elif flag==1 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_2 = dense_size_1
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_2 = 64
                                                                                    dense_sizes_1 = [opt_layer_2]

                                                                                elif flag==2 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_3 = dense_size_2
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_3 = 64
                                                                                    dense_sizes_2 = [opt_layer_3]

                                                                                elif flag==3 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_4 = s1_filter_size_1
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_4 = 16
                                                                                    s1_filter_sizes_1 = [opt_layer_4]
                                                                                    
                                                                                elif flag==4 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_5 = s2_filter_size_1
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_5 = 16
                                                                                    s2_filter_sizes_1 = [opt_layer_5]
                                                                                    
                                                                                elif flag==5 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_6 = s2_filter_size_2
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_6 = 16
                                                                                    s2_filter_sizes_2 = [opt_layer_6]
                                                                                    
                                                                                elif flag==6 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_7 = s2_filter_size_3
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_7 = 128
                                                                                    s2_filter_sizes_3 = [opt_layer_7]

                                                                                elif flag==7 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_8 = s3_filter_size_1
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_8 = 16
                                                                                    s3_filter_sizes_1 = [opt_layer_8]
                                                                                    
                                                                                elif flag==8 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_9 = s3_filter_size_2
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_9 = 16
                                                                                    s3_filter_sizes_2 = [opt_layer_9]
                                                                                    
                                                                                elif flag==9 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_10 = s3_filter_size_3
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_10 = 128
                                                                                    s3_filter_sizes_3 = [opt_layer_10]
                                                                                    
                                                                                elif flag==10 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_11 = s4_filter_size_1
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_11 = 16
                                                                                    s4_filter_sizes_1 = [opt_layer_11]
                                                                                    
                                                                                elif flag==11 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_12 = s4_filter_size_2
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_12 = 16
                                                                                    s4_filter_sizes_2 = [opt_layer_9]
                                                                                    
                                                                                elif flag==12 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_13 = s4_filter_size_3
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_13 = 128
                                                                                    s4_filter_sizes_3 = [opt_layer_13]
                                                                                    
                                                                                elif flag==13 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_14 = s5_filter_size_1
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_14 = 16
                                                                                    s5_filter_sizes_1 = [opt_layer_14]
                                                                                    
                                                                                elif flag==14 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_15 = s5_filter_size_2
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_15 = 16
                                                                                    s5_filter_sizes_2 = [opt_layer_15]
                                                                                    
                                                                                elif flag==15 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_16 = s5_filter_size_3
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_16 = 128
                                                                                    s5_filter_sizes_3 = [opt_layer_16]
                                                                                    
                                                                                elif flag==16 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_17 = dp_size_1
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_17 = 0.0
                                                                                    dp_sizes_1 = [opt_layer_17]
                                                                                    
                                                                                elif flag==17 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_18 = dp_size_2
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_18 = 0.0
                                                                                    dp_sizes_2 = [opt_layer_18]
                                                                                    
                                                                                elif flag==18 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_19 = wd_size_1
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_19 = 0.0
                                                                                    wd_sizes_1 = [opt_layer_19]
                                                                                    
                                                                                elif flag==19 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_20 = wd_size_2
                                                                                        temp=test_acc
                                                                                    else :
                                                                                        opt_layer_20 = 0.0
                                                                                    wd_sizes_2 = [opt_layer_20]
                                                                            flag=1
                                                                            test_acc=0
                                                                        flag=2
                                                                        test_acc=0
                                                                    flag=3
                                                                    test_acc=0
                                                                flag=4
                                                                test_acc=0
                                                            flag=5
                                                            test_acc=0
                                                        flag=6
                                                        test_acc=0
                                                    flag=7
                                                    test_acc=0
                                                flag=8
                                                test_acc=0
                                            flag=9
                                            test_acc=0
                                        flag=10
                                        test_acc=0  
                                    flag=11
                                    test_acc=0
                                flag=12
                                test_acc=0
                            flag=13
                            test_acc=0
                        flag=14
                        test_acc=0
                    flag=15
                    test_acc=0
                flag=16
                test_acc=0
            flag=17
            test_acc=0
        flag=18
        test_acc=0
    flag=19
    test_acc=0
    
print("######################################################################################################")
print("Trials: ",combinations)
print("Optimal Validation Accuracy",temp)
print("The Final Optimal Values for all block is : ")
print("learning rate: ",opt_layer_1,"\ndense_size_1: ",opt_layer_2,"\ndense_size_2: ",opt_layer_3,"\ns1_filter_size_1: ",opt_layer_4,"\ns2_filter_size_1: ",opt_layer_5,"\ns2_filter_size_2: ",opt_layer_6,"\ns2_filter_size_3: ",opt_layer_7,"\ns3_filter_size_1: ",opt_layer_8,"\ns3_filter_size_2: ",opt_layer_9,"\ns3_filter_size_3: ",opt_layer_10,"\ns4_filter_size_1: ",opt_layer_11,"\ns4_filter_size_2: ",opt_layer_12,"\ns4_filter_size_3: ",opt_layer_13,"\ns5_filter_size_1: ",opt_layer_14,"\ns5_filter_size_2: ",opt_layer_15,"\ns5_filter_size_3: ",opt_layer_16,"\ndp_size_1: ",opt_layer_17,"\ndp_size_2: ",opt_layer_18,"\nwd_size_1: ",opt_layer_19,"\nwd_size_2: ",opt_layer_20)                      

0.1 64 64 16 16 16 128 16 16 128 16 16 128 16 16 128 0.0 0.0 0.0 0.0
313/313 [==============================] - 16s 31ms/step - loss: 2.7289 - accuracy: 0.1015 - val_loss: 2.3084 - val_accuracy: 0.0946
0.09459999948740005 0
0.01 64 64 16 16 16 128 16 16 128 16 16 128 16 16 128 0.0 0.0 0.0 0.0
313/313 [==============================] - 14s 30ms/step - loss: 2.2167 - accuracy: 0.1635 - val_loss: 2.0845 - val_accuracy: 0.2038
0.2037999927997589 0.09459999948740005
0.001 64 64 16 16 16 128 16 16 128 16 16 128 16 16 128 0.0 0.0 0.0 0.0
313/313 [==============================] - 14s 30ms/step - loss: 2.3180 - accuracy: 0.1385 - val_loss: 2.2224 - val_accuracy: 0.1470
0.1469999998807907 0.2037999927997589
0.0001 64 64 16 16 16 128 16 16 128 16 16 128 16 16 128 0.0 0.0 0.0 0.0
313/313 [==============================] - 14s 29ms/step - loss: 2.5476 - accuracy: 0.1024 - val_loss: 2.3103 - val_accuracy: 0.0940
0.09399999678134918 0.2037999927997589
1e-05 64 64 16 16 16 128 16 16 128 16 16 128 16 

0.21240000426769257 0.29679998755455017
0.01 64 64 512 16 512 128 512 16 128 16 16 128 16 16 128 0.0 0.0 0.0 0.0
313/313 [==============================] - 17s 38ms/step - loss: 2.1586 - accuracy: 0.1744 - val_loss: 2.1920 - val_accuracy: 0.1678
0.16779999434947968 0.29679998755455017
0.01 64 64 512 16 512 128 16 32 128 16 16 128 16 16 128 0.0 0.0 0.0 0.0
313/313 [==============================] - 16s 37ms/step - loss: 2.1734 - accuracy: 0.1778 - val_loss: 2.1163 - val_accuracy: 0.2436
0.243599995970726 0.29679998755455017
0.01 64 64 512 16 512 128 16 64 128 16 16 128 16 16 128 0.0 0.0 0.0 0.0
313/313 [==============================] - 17s 37ms/step - loss: 2.1322 - accuracy: 0.1961 - val_loss: 2.1923 - val_accuracy: 0.2640
0.2639999985694885 0.29679998755455017
0.01 64 64 512 16 512 128 16 128 128 16 16 128 16 16 128 0.0 0.0 0.0 0.0
313/313 [==============================] - 17s 37ms/step - loss: 2.1803 - accuracy: 0.1813 - val_loss: 2.0089 - val_accuracy: 0.2544
0.25440001487731934 0